In [ ]:
import wmfdata
import urllib.parse
import pyspark.sql.functions as sf
import pandas as pd

In [ ]:
spark = wmfdata.spark.create_session(
    type='yarn-large',
    app_name='htriedman sparql query extraction',
    ship_python_env=True
)

In [ ]:
query = """
SELECT e.el_from as page_id, e.el_to_path as url_encoded_query
FROM wmf_raw.mediawiki_externallinks e
WHERE
    e.el_to_domain_index = 'https://org.wikidata.query.'
    AND e.el_to_path != '/'
    AND e.wiki_db = 'wikidatawiki'
    AND e.snapshot = '2023-12'
"""

In [ ]:
df = spark.sql(query)

In [ ]:
decode = sf.udf(urllib.parse.unquote)

In [ ]:
df = df.withColumn('decoded_query', decode(df.url_encoded_query))

In [ ]:
pattern = r'^/##(?!defaultView:)|#title:'

# Filtering the DataFrame
filtered_df = df.filter(df.decoded_query.rlike(pattern))

In [ ]:
pd_df = filtered_df.toPandas()

In [ ]:
pd_df = pd_df[['page_id', 'decoded_query']]

In [ ]:
pd_df['decoded_query'] = pd_df['decoded_query'].str[2:]

In [ ]:
print(pd_df['decoded_query'][500])

In [ ]:
pd_df.to_pickle('wikidata-sparql-titles.pkl')

## Start investigating link context

In [ ]:
pd_df = pd.read_pickle('wikidata-sparql-titles.pkl')

In [ ]:
pd_df

In [ ]:
spark.sql("""
select *
from wmf.mediawiki_wikitext_current
where snapshot = '2023-12'
and wiki_db = 'wikidatawiki'
and page_id = 22470558
""").show()

In [ ]:
spark.sparkContext.stop()
spark.stop()  